Jan. 17: initial notebook set up

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

#Just listing the first 5 .dcm's in the sample images file

In [ ]:
import dicom
import os

PathDicom = '../input/sample_images'
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))
print(lstFilesDCM[:5])

#Gathering more DICOM information from one .dcm file.......

In [ ]:
import dicom
dataset = dicom.read_file('../input/sample_images/0d941a3ad6c889ac451caf89c46cb92a/8d7f0315634af2cfda1aa47f12a8e954.dcm')
print("Storage type  ",dataset.SOPClassUID)
#print("Patient's name...:", display_name)- obviously not included
print("Patient id.......:", dataset.PatientID)
print("Modality.........:", dataset.Modality)
#print("Study Date.......:", dataset.StudyDate)
#print("Rescale Type.........:", float(dataset.RescaleType))
print("Rescale Slope.........:", str(dataset.RescaleSlope))
print("Rescale Intercept.........:", str(dataset.RescaleIntercept))
print("Pixel Spacing.........:", str(dataset.PixelSpacing))

if 'PixelData' in dataset:
    rows = int(dataset.Rows)
    cols = int(dataset.Columns)
    print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
        rows=rows, cols=cols, size=len(dataset.PixelData)))
    if 'PixelSpacing' in dataset:
        print("Pixel spacing....:", dataset.PixelSpacing)

#Obtain tags from DICOM file, plot one image
So let's see what the pixel array looks like for one .dcm file (slice),  print all DICOM tags, and plot an image (of the array​). Tag information will become more import in future iterations.

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import dicom
import numpy as np

dataset = dicom.read_file('../input/sample_images/0d941a3ad6c889ac451caf89c46cb92a/8d7f0315634af2cfda1aa47f12a8e954.dcm')
img=dataset.pixel_array

print(img) #first just show the array
print('')
print(dataset) #Then the dicom file tags

imgplot = plt.imshow(img)

See what OpenCV can do.....


#Let's just plot an image to grayscale...

In [ ]:
import numpy as np
import dicom
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt



dataset = dicom.read_file('../input/sample_images/0d941a3ad6c889ac451caf89c46cb92a/8d7f0315634af2cfda1aa47f12a8e954.dcm')
img=dataset.pixel_array

plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis

img = Image.fromarray(img)
img.save("8d7f0315634af2cfda1aa47f12a8e954.png")
plt.show()

#Clustering...incomplete

In [ ]:
#lifted from http://scikit-learn.org/stable/auto_examples/cluster/plot_color_quantization.html

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
from time import time
import dicom

n_colors = 64 #reduce amount here....

dataset = dicom.read_file('../input/sample_images/0d941a3ad6c889ac451caf89c46cb92a/8d7f0315634af2cfda1aa47f12a8e954.dcm')
img=dataset.pixel_array


#= np.array(china, dtype=np.float64) / 255

# Load Image and transform to a 2D numpy array.


print("Fitting model on a small sub-sample of the data")
t0 = time()
image_array_sample = shuffle(image_array, random_state=0)[:1000]
kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)
print("done in %0.3fs." % (time() - t0))

# Get labels for all points
print("Predicting color indices on the full image (k-means)")
t0 = time()
labels = kmeans.predict(image_array)
print("done in %0.3fs." % (time() - t0))


codebook_random = shuffle(image_array, random_state=0)[:n_colors + 1]
print("Predicting color indices on the full image (random)")
t0 = time()
labels_random = pairwise_distances_argmin(codebook_random,
                                          image_array,
                                          axis=0)
print("done in %0.3fs." % (time() - t0))


def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image

# Display all results, alongside original image
plt.figure(1)
plt.clf()
ax = plt.axes([0, 0, 1, 1])
plt.axis('off')
plt.title('Original image (96,615 colors)')
plt.imshow(china)

plt.figure(2)
plt.clf()
ax = plt.axes([0, 0, 1, 1])
plt.axis('off')
plt.title('Quantized image (64 colors, K-Means)')
plt.imshow(recreate_image(kmeans.cluster_centers_, labels, w, h))

plt.figure(3)
plt.clf()
ax = plt.axes([0, 0, 1, 1])
plt.axis('off')
plt.title('Quantized image (64 colors, Random)')
plt.imshow(recreate_image(codebook_random, labels_random, w, h))
plt.show()
